# burnin

> Fill in a module description here

In [ ]:
#| default_exp burnin

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import jax
import jax.numpy as jnp
from jaxtyping import Array, Float
from typing import Optional

from chewc.sp import SimParam
from chewc.population import Population
from chewc.phenotype import set_pheno
from chewc.pipe import run_generation

def run_burnin(
    key: jax.random.PRNGKey,
    sp: SimParam,
    n_generations: int,
    n_parents: int,
    n_progeny: int,
    h2: Float[Array, "nTraits"],
    verbose: bool = True
) -> Population:
    """
    Executes a multi-generational burn-in phase using random mating and
    truncation selection.

    This function takes an initial founder population (defined in `sp.founderPop`)
    and iterates for a specified number of generations. In each generation, it
    selects the top parents based on their phenotype and randomly crosses them
    to produce the next generation. The entire per-generation logic is JIT-compiled
    for maximum performance.

    Args:
        key: The base JAX random key for the entire burn-in simulation.
        sp: A fully configured SimParam object, which must include traits and a
            founder population.
        n_generations: The number of burn-in generations to simulate.
        n_parents: The number of top individuals to select as parents each generation.
        n_progeny: The number of progeny (crosses) to create each generation.
        h2: The heritability to use for phenotyping in each generation.
        verbose: If True, prints the mean phenotype for each generation.

    Returns:
        The final Population object after the last burn-in generation.
    """
    if sp.founderPop is None or sp.traits is None:
        raise ValueError("SimParam must have a founder population and traits defined before running burn-in.")

    if verbose:
        print(f"--- Starting Accelerated Burn-in ({n_generations} Generations) ---")

    # --- 1. Set Initial Phenotypes for the Founder Population ---
    key, pheno_key = jax.random.split(key)
    current_pop = set_pheno(
        key=pheno_key,
        pop=sp.founderPop,
        traits=sp.traits,
        ploidy=sp.ploidy,
        h2=h2
    )

    # --- 2. JIT-Accelerated Burn-in Loop ---
    # The main loop is standard Python, but the function it calls (`run_generation`)
    # is JIT-compiled. The first call will be slower due to compilation.
    for gen in range(n_generations):
        key, generation_key = jax.random.split(key)

        current_pop = run_generation(
            key=generation_key,
            pop=current_pop,
            h2=h2,
            n_parents=n_parents,
            n_crosses=n_progeny,
            # Pass static arguments required by the JIT-compiled function
            use_pheno_selection=True,
            select_top_parents=True,
            ploidy=sp.ploidy,
            # Pass static SimParam components directly
            gen_map=sp.gen_map,
            recomb_param_v=sp.recomb_params[0],
            traits=sp.traits
        )

        if verbose:
            # Block until computation is finished to get the correct mean phenotype
            # for this generation before printing.
            mean_pheno = jnp.mean(current_pop.pheno[:, 0]).block_until_ready()
            print(f"Generation {gen + 1:2d}/{n_generations} | Mean Phenotype: {mean_pheno:.4f}")

    if verbose:
        print("\n--- Burn-in Complete ---")

    return current_pop








In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()